In [1]:
#!pip install implicit

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    recs = np.random.choice(items_weights['item_id'], size=n, replace=False, p=items_weights['weight'])
    return recs.tolist()

In [6]:
items_prob = (data.groupby('item_id')['quantity'].sum().reset_index()).rename(columns={"quantity": "weight"})
items_prob.head()

,item_id,weight
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [7]:
items_prob['weight'] = items_prob['weight']/items_prob['weight'].sum()
# check weight/probability sum = 1
items_prob['weight'].sum()

1.0

In [8]:
%%time

weighted_random_recommendation(items_prob, 2)

CPU times: user 3.71 ms, sys: 2.13 ms, total: 5.84 ms
Wall time: 6.08 ms


[6534178, 6534166]

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [9]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[553111, 990130, 14025228, 8091299, 855288]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[8155825, 1841061, 946817, 1541203, 8203466]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [10]:
result.columns[1:]

Index(['actual', 'random_recommendation', 'popular_recommendation', 'itemitem',
       'cosine', 'tfidf', 'own_purchases'],
      dtype='object')

In [11]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
    
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant

def map_k(recommended_lists, bought_lists, k=5):
    
    if np.shape(recommended_lists)[0] == np.shape(bought_lists)[0]:
        result_list = np.sum([ap_k(recommended_list_i, bought_list_i, k) \
                         for recommended_list_i, bought_list_i in zip(recommended_lists, \
                                                                      bought_lists)])/ \
                np.shape(recommended_lists)[0]
    else:
        result_list = False
    return result_list

In [12]:
# lets convert object data to integers' list

def str_to_int_list_converter(data):
    data = list(map(lambda item: item.replace(',', ""), data))
    return list(map(lambda item: list(map(lambda item_id: int(item_id) if item_id.isdigit() else None, item[1:-1].split())) , data))

In [13]:
for column in result.columns[1:]:
    result[column] = str_to_int_list_converter(result[column])

In [14]:
def precision_calc(df, k = 5):
    """
    last df column - bought items, first - user id, intermediate - recommended items by different algorithms
    """
   
    bought_items = df[df.columns[-1]]
    data = np.array([])
    
    for column in df.columns[1:-1]:
        alg_recommended_list = df[column]
        data = np.append(data, [map_k(alg_recommended_list, bought_items, k)])
    
    output_df = pd.DataFrame([data], columns=df.columns[1:-1])
    
    return output_df

In [15]:
result.iloc[1]

user_id                                                                   3
actual                    [835476, 851057, 872021, 878302, 879948, 90963...
random_recommendation          [8155825, 1841061, 946817, 1541203, 8203466]
popular_recommendation        [6534178, 6533889, 1029743, 6534166, 1082185]
itemitem                         [999999, 1082185, 981760, 1098066, 995242]
cosine                           [1082185, 1098066, 981760, 999999, 826249]
tfidf                            [1082185, 981760, 1098066, 826249, 999999]
own_purchases                 [1082185, 1098066, 6534178, 1127831, 1068719]
Name: 1, dtype: object

In [16]:
for k in range(1,6):
    if k == 1:
        precisions_alg_s = precision_calc(result, k)
    else:
        precisions_alg_s = precisions_alg_s.append(precision_calc(result, k), ignore_index=True)

In [17]:
# index - k-1
precisions_alg_s

,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf
0,0.046033,0.0,0.543095,0.000000,0.842311,0.835945
1,0.054603,0.0,0.543095,0.422625,0.845250,0.843781
2,0.062235,0.0,0.564887,0.422625,0.840393,0.799543
3,0.065948,0.0,0.564887,0.435480,0.721433,0.748408
4,0.069234,0.0,0.529046,0.440638,0.703285,0.732791


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [18]:
# your_code